In [1]:
suppressPackageStartupMessages({
    library(Seurat)
    library(tools) 
    library(harmony)
    library(AnnotationDbi)
    library(org.Hs.eg.db)  
    library(RCurl)
    library(dplyr)
    library(AnnotationHub)
    library(parallel)
    library(future)
})

### Load cycle scoring functions from Utility dir
- Scaling data using cell cycle score helps in avoiding bias due to difference in cell cycle states, and enables us to integrate the cells more accurately

In [2]:
source("../Utility/Matrix.utils.R") 
source("../Utility/cell_cycle_scoring.r")

In [3]:
ndim = 50
nfeat = 2000
nmax = 5000

In [4]:
seurat_list <- readRDS("../Step3_Preprocessing/out/seurat_list_filtered.rds")

In [ ]:
for (name in names(seurat_list)) {
    print(unique(seurat_list[[name]]@meta.data$orig.ident))
}

In [ ]:
log_conn <- file("./out/log_file_scaling.txt", open = "wt")
sink(log_conn, append = FALSE)
sink(log_conn, type = "message")

for (sample_name in names(seurat_list)) {
    seurat_sample <- seurat_list[[sample_name]]
    DefaultAssay(seurat_sample) <- "RNA"
    if (nrow(seurat_sample) <= 50) {
        next
    }
    
    seurat_sample <- NormalizeData(seurat_sample, verbose = FALSE)
    seurat_sample <- suppressWarnings(FindVariableFeatures(
                                    seurat_sample,
                                    selection.method = "vst",
                                    nfeatures = nfeat,
                                    verbose = FALSE
                    ))
    seurat_sample <- suppressWarnings(cell_cycle_scoring(seurat_sample, cc_file="../Utility/Data/CC_Homo_sapiens.csv"))
    seurat_sample <- ScaleData(
                        seurat_sample,
                        vars.to.regress = "CC.Difference", 
                        features = rownames(seurat_sample),
                        verbose = FALSE
                    )
    seurat_sample <- RunPCA(seurat_sample, npcs = ndim, features = VariableFeatures(seurat_sample), verbose = FALSE)
    seurat_list[[sample_name]] <- seurat_sample
}

sink(type = "message")
sink()
close(log_conn)

In [ ]:
combined <- merge(
    x = seurat_list[[1]],
    y = seurat_list[-1],
    add.cell.ids = names(seurat_list),
    project = "HarmonyIntegration"
)
combined$filename <- combined$orig.ident
Idents(combined) <- combined$orig.ident
combined <- JoinLayers(combined)

In [ ]:
combined <- NormalizeData(combined, verbose = FALSE)
combined <- FindVariableFeatures(
                        combined, 
                        selection.method = "vst", 
                        nfeatures = nfeat, 
                        verbose = FALSE
            )

combined <- cell_cycle_scoring(combined, cc_file="../Utility/Data/CC_Homo_sapiens.csv")
all.genes <- rownames(seurat_objects[[name]])
combined <- ScaleData(combined, features=all.genes)
combined <- RunPCA(combined, npcs = ndim, verbose = FALSE)

In [ ]:
combined <- RunHarmony(
    object = combined,
    group.by.vars = "orig.ident",
    assay.use = "RNA",
    reduction.use = "pca",
    dims.use = 1:ndim,
    verbose = TRUE
)

In [ ]:
combined <- RunUMAP(combined, reduction = "harmony", dims = 1:ndim)
DimPlot(combined, reduction="umap", group.by="orig.ident")

### Save data for downstream analysis

In [ ]:
saveRDS(combined, file = file.path("./out/integrated_harmony.rds"))